# Course: **COMP 333 — Final Project Phase 1**
**Group:** O <br>
**Names:**
- Carson Johnston - **Student ID:** 40312846
- Charlotte Lauzon - **Student ID:** 40285642
- Ava Samimi - **Student ID:** 40048117

## Description of the task:
The objective of Phase 1 of this project is to select a large, real world dataset (>= 1GB). We then do the data retrieval, the wrangling, an EDA with 2 research questions, a baseline model and a report on Jupyter Notebook.

## Source of the datasets:
The **US Used Cars** dataset is a public dataset available on Kaggle: <br> https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset

## Description of the datasets:
The **US Used Cars** dataset exceeds the minimum size requirements, has a large number of features, includes real world inconsistencies and supports both supervised and unsupervised learning tasks.

It contains detailed information about 3 million used cars details across the United States.<br>
Each row represents a single vehicle listing, wth each column being an attribute.

**The *US Used Cars* dataset contains:**
- Vehicle Identification (Vehicle identification number, listing ID)
- Vehicle Specifications (Engines, horsepower, fuel, size, legroom, bed, body...)
- Market/Dealer Information (city, ZIP, position, days on market, date of listing)
- Fuel efficiency
- Condition and History (accidents, damages, if it was a cab, if it is new...)
- Categorical and Descriptive Features (colours, description, picture...)

The dataset has a mix of numerical, boolean, high-cardinality categorical and string variables

**The notebook will include: **
1. Data Retrieval
2. Wrangling/Cleaning
3. EDA
4. Baseline Model
5. Report


In [1]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.gridspec as gridspec
import warnings

# Avoid unnecessary warnings in output
#warnings.filterwarnings("ignore")

# Doesn't cut tables
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

# Apply predefined visual theme for consistence in the export
plt.style.use('seaborn-v0_8')
sns.set_theme()

## 1. Data Retrieval: Phase 1 – Step 1

### 1.1 Source
We are using the **US Used Cars** public dataset from Kaggle:
https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset

This dataset is downloaded as a csv file, and has a size of 9.98 Go

### 1.2 Retrieval
The dataset is retrieved as a **file-based source csv** and is loaded into Python using the pandas function (`read_csv`).
We retrieved the data as a CSV file because the data is structured and tabular, it is delivered by Kaggle as a flat file.

Kaggle provides the dataset as a downloadable csv file. Since the dataset is static, the most reproducible and efficient approach is to store the raw file locally and load it consistently.

### 1.3 Challenge Handling
- **Large file size:** Because the Data file is nearly 10 Go, the direct loading can be slow and memory-heavy.
  - Mitigation: we use `low_memory=False`. This makes sure the entire file is read before deciding the columns' data type, which leads to less dtypewarning
- **Authentication / rate limits:** not applicable because we are not calling an API endpoint in this phase (download is done once via Kaggle).
  - If we use Kaggle API later, authentication would require a Kaggle API token. However, for this phase, one static file download was more efficient and safe.

### 1.4 Raw Data Storage
The dataset is stored in the `data/raw/` folder and is treated as read-only.

In [2]:
import os
raw_data_path = "data/raw/used_cars_data.csv"

# Load the raw dataset
cars = pd.read_csv(raw_data_path, low_memory=False)

print("Shape:", cars.shape)
cars.head()

Shape: (3000040, 66)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [3]:
# TEST DATASET

raw_data_path = "data/raw/used_cars_data.csv"

# For first 1000 rows
testing_cars = pd.read_csv(raw_data_path, nrows=1000, low_memory=False)
print("Shape:", testing_cars.shape)
testing_cars.head()

Shape: (1000, 66)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


## 2. Wrangling/Cleaning: Phase 1 – Step 2

We perform an initial data audit:
- Missing value analysis
- Duplicate detection
- Outlier detection
- Data type validation
- Reproducible cleaning pipeline

We use the `quantDDA()` and `vizDDA()` functions from Lab 2 to standardize our audit process.

In [ ]:
# The function quantDDA(df) takes a pandas DataFrame as argument
# The function returns a new DataFrame with a summary of each column
def quantDDA(df):

# create a list to put all column's summary in
    summary_list = []

# Loop through all columns of the argument dataset
    for column in df.columns:
        # One column is a series, we create a dictionary to store the statistics
        series = df[column]
        summary = {}

        # Store the column name as the feature
        summary["Feature"] = column
        # Store the number of rows of the database
        summary["Number of Observations"] = len(series)
        # Store the number of only valid, non-missing values
        summary["Number of Entries"] = series.count()
        # Store the number of distinct values only
        summary["Number of Unique Entries"] = series.nunique()
        # Store the number of missing values (where its n/a is "true")
        summary["Number of Missing Entries"] = series.isna().sum()

        # Store the mode values as a list
        modes = series.mode()
        summary["Mode(s)"] = list(modes)

        # The categorical columns do not have all the numeric fields;
        # To make sure the table has no issues with columns, we set as NaN
        summary["Number of Outliers (IQR)"] = np.nan
        summary["Number of Extreme Values (top/bottom 1%)"] = np.nan
        summary["Mean"] = np.nan
        summary["Standard Deviation"] = np.nan
        summary["Maximum"] = np.nan
        summary["Minimum"] = np.nan
        summary["Q1"] = np.nan
        summary["Q2 (Median)"] = np.nan
        summary["Q3"] = np.nan
        summary["Skewness"] = np.nan
        summary["Kurtosis"] = np.nan

        # If its numerical column, compute numeric statistics
        # Otherwise, skip the calculations
        if pd.api.types.is_numeric_dtype(series):
            # We remove missing values to not compute with invalid numbers
            # pandas ignore NaN, but not SciPy, so we drop beforehand
            x = series.dropna()

            # Store all the numerical statistics
            if len(x) > 0:
                summary["Mean"] = x.mean()
                summary["Standard Deviation"] = x.std()
                summary["Maximum"] = x.max()
                summary["Minimum"] = x.min()
                summary["Q1"] = x.quantile(0.25)
                summary["Q2 (Median)"] = x.median()
                summary["Q3"] = x.quantile(0.75)

                # Number of outliers with IQR Method
                IQR = summary["Q3"] - summary["Q1"]
                lower = summary["Q1"] - 1.5 * IQR
                upper = summary["Q3"] + 1.5 * IQR
                # Count values outside the lower and upper bounds
                outliers = x[(x < lower) | (x > upper)]
                summary["Number of Outliers (IQR)"] = outliers.count()

                # Extreme values (Top and Bottom 1%)
                lower_ext = x.quantile(0.01)
                upper_ext = x.quantile(0.99)
                # Counts the number of extreme values of the top and bottom 1%
                extreme = x[(x < lower_ext) | (x > upper_ext)]
                summary["Number of Extreme Values (top/bottom 1%)"] = extreme.count()

                # Use SciPy for skewness and kurtosis
                summary["Skewness"] = stats.skew(x, bias=False)
                summary["Kurtosis"] = stats.kurtosis(x, fisher=True, bias=False)

        # Add the dictionary to list
        summary_list.append(summary)

    result = pd.DataFrame(summary_list)

    # Make sure the columns are in the same order as in assignment
    ordered_columns = [
        "Feature",
        "Number of Observations",
        "Number of Entries",
        "Number of Unique Entries",
        "Number of Missing Entries",
        "Number of Outliers (IQR)",
        "Number of Extreme Values (top/bottom 1%)",
        "Mode(s)",
        "Mean",
        "Standard Deviation",
        "Maximum",
        "Minimum",
        "Q1",
        "Q2 (Median)",
        "Q3",
        "Skewness",
        "Kurtosis"
    ]

    return result[ordered_columns]

In [ ]:
# The function vizDDA(df) takes a pandas DataFrame as argument
# The function produces a square grid of plots (univariate on diagonal, bivariate off diagonal),
# and a heatmap of the missing values.
def vizDDA(df):

    df = df.copy()

    # Remove high-cardinality categorical columns for visualization grid
    high_cardinality_cols = []

    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            # put an arbitrary 20 threshold
            if df[col].nunique() > 20:
                high_cardinality_cols.append(col)
    # df for the plots
    df_plot = df.drop(columns=high_cardinality_cols)

    num_features = len(df_plot.columns)

    # Detect datetime columns
    is_datetime = {
        column: pd.api.types.is_datetime64_any_dtype(df_plot[column])
        for column in df_plot.columns
    }

    # Create square grid
    # put 4x4 inch space for each subplot for readability
    fig = plt.figure(figsize=(4 * num_features, 4 * num_features))
    grid = gridspec.GridSpec(num_features, num_features)

    # Consider i as index for rows, and j as index for columns
    for i, col_i in enumerate(df_plot.columns):
        for j, col_j in enumerate(df_plot.columns):

            ax = fig.add_subplot(grid[i, j])

            # Detect if its numeric or categorical
            is_num_i = pd.api.types.is_numeric_dtype(df_plot[col_i])
            is_num_j = pd.api.types.is_numeric_dtype(df_plot[col_j])

            # Univariate on diagonal
            # if we compare a feature with itself, we are doing a univariate analysis
            if i == j:

                # If it's a datetime univariate, we count how many times each unique datetime appear
                if is_datetime[col_i]:
                    series = df_plot[col_i].dropna()
                    if len(series) > 0:
                        series.value_counts().sort_index().plot(ax=ax)
                    ax.set_ylabel("count")

                # if numeric, we do a histogram to visualize the distribution
                elif is_num_i:
                    sns.histplot(df_plot[col_i], kde=True, ax=ax)

                # For categorical univariate, we do a bar chart
                else:
                    df_plot[col_i].value_counts().plot(kind="bar", ax=ax)

            # Bivariate off diagonal
            # We study two different variables
            else:

                # If one variable is datetime, and the other is numeric:
                # We do a line plot
                # Datetime vs Numeric when datetime is the row variable, and numeric is column variable
                if is_datetime[col_i] and is_num_j:
                    # Select the relevant columns, drop the missing values, and sort by the datetime column
                    tmp = df_plot[[col_i, col_j]].dropna().sort_values(col_i)
                    # We draw datetime on x-axis, numeric on y-axis
                    ax.plot(tmp[col_i], tmp[col_j])

                # Same but for the symmetric situation;
                # Datetime vs Numeric when datetime is the column variable, and numeric is row variable
                elif is_datetime[col_j] and is_num_i:
                    tmp = df_plot[[col_j, col_i]].dropna().sort_values(col_j)
                    ax.plot(tmp[col_j], tmp[col_i])

                # If both variables are numeric (Numeric vs numeric), we do a scatter plot
                elif is_num_i and is_num_j:
                    sns.scatterplot(x=df_plot[col_j], y=df_plot[col_i], ax=ax)

                # If there are mixed types (categorical vs numeric) we do a boxplot
                # For when numerical is the row feature and categorical is the column feature
                elif is_num_i and not is_num_j:
                    sns.boxplot(x=df_plot[col_j], y=df_plot[col_i], ax=ax)

                # For when numerical is the column feature and categorical is the row feature
                elif not is_num_i and is_num_j:
                    sns.boxplot(x=df_plot[col_i], y=df_plot[col_j], ax=ax)

                # If both variable are categorical (Categorical vs categorical), we do a grouped bar chart
                # Since all cases with numerical variables have been checked,
                # only categorical on both row and colum left
                else:
                    categorical = pd.crosstab(df_plot[col_i], df_plot[col_j])
                    categorical.plot(kind="bar", ax=ax)

            # Axis labeling
            # Only label bottom row with x labels
            if i == num_features - 1:
                ax.set_xlabel(col_j, fontsize=15, fontweight="bold")
            else:
                ax.set_xlabel("")

            # Only label left column with y labels
            if j == 0:
                ax.set_ylabel(col_i, fontsize=15, fontweight="bold")
            else:
                ax.set_ylabel("")

    # Add title
    fig.suptitle("Visualization Grid (Univariate on Diagonal, Bivariate Off-Diagonal)", fontsize=25, fontweight="bold")
    # Improve spacing so subplots don't overlap
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # Heatmap of missing values
    plt.figure(figsize=(12, 6))
    sns.heatmap(df.isna(), cbar=True, vmin=0, vmax=1)
    plt.title("Missing Values Heatmap")
    plt.show()

In [ ]:
quantDDA(cars)

EXPLANATION OF AUDIT


THEN FROM WEEK 7: missing value, duplicate, outlier, normalize, reproducable cleaning pipeline

In [ ]:
# Stuff to add here for everything

In [ ]:
#Create function for pipelin
def clean_cars(df):

    df = df.copy()

    return df

In [ ]:
cars_clean = clean_cars(cars)